In [2]:
import asyncio
import requests
import re
from bs4 import BeautifulSoup
import time
from math import ceil
import os
DIR = os.getenv('KEA_BASE_DIR')

In [3]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51',
          'Accept':"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"}

def http_get_sync(url: str):
    response = requests.get(url,headers=headers,timeout=(15.0,30.0))
    return response.content
    
async def http_get(url: str):
    return await asyncio.to_thread(http_get_sync, url)

In [4]:
response = requests.get("https://jds.ind.in/en/format-c7-of-candidates-ac-2023.php",headers=headers,timeout=(15.0,30.0))

In [5]:
soup = BeautifulSoup(response.text,'html.parser')

def has_link(href):
    return href and bool(re.search(r"/candidates-case-pdf/",href))
    
temp_list = soup.find_all('a',href=has_link)
documents = list(set(temp_list))

In [9]:
for document in documents:
    url = "https://jds.ind.in" + document['href'].strip('.')
    pdf = await http_get(url)
    name = document['href'].strip('.').replace('/candidates-case-pdf/','').replace('.docx','')
    with open(f"{DIR}/2023 Elections/Format C7/JDS Format C7/{name}",'wb') as file:
        file.write(pdf)